In [2]:
import pandas as pd
import useful_functions as use
import pyodbc

In [68]:
PPD = use.get_ppd()

INFO:useful_functions:PPD file from Jan 22 2022


In [42]:
PPD['IQVIA_ME'] = [x[0:10] for x in PPD.ME]

In [43]:
PPD = pd.merge(PPD, npi, on='ME')

In [44]:
familty = PPD[PPD.PRIM_SPEC_CD=='FM']

In [45]:
internists = PPD[PPD.PRIM_SPEC_CD=='IM']

In [8]:
#iqvia
username = 'vigrose'
password = 'Hufflepuff~10946'
q = "DSN=eprdods; UID={}; PWD={}".format(username, password)
ODS = pyodbc.connect(q)
iqvia_query = \
        """
        SELECT DISTINCT 
        B.BUSINESS_NAME,
        B.COT_CLASSIFICATION,
        B.COT_FACILITY_TYPE,
        B.COT_SPECIALTY,
        A.AFFIL_TYPE_ID,
        A.AFFIL_RANK,
        P.ME
        FROM 
        ODS.ODS_IMS_BUSINESS B, ODS.SAS_ODS_IMS_PROVIDER_AFFIL A, ODS.ODS_IMS_PROFESSIONAL P
        WHERE  
        B.IMS_ORG_ID = A.IMS_ORG_ID
        AND
        A.PROFESSIONAL_ID = P.PROFESSIONAL_ID
        AND
        P.CURRENT_BATCH_FLAG='Y'
        AND
        A.CURRENT_BATCH_FLAG='Y'
        AND
        B.CURRENT_BATCH_FLAG='Y'
        """
iqvia = pd.read_sql(con=ODS, sql=iqvia_query)
iqvia.head()

,BUSINESS_NAME,COT_CLASSIFICATION,COT_FACILITY_TYPE,COT_SPECIALTY,AFFIL_TYPE_ID,AFFIL_RANK,ME
0,LARGO MEDICAL CENTER,Hospital,Acute Care Hospital,General Acute,3.0,6,0010200001
1,SOUTH BALDWIN REGIONAL MEDICAL CENTER,Hospital,Acute Care Hospital,General Acute,3.0,6,0010200001
2,MISSION HEALTH,Hospital,Acute Care Hospital,General Acute,3.0,6,0010200003
3,"MISSION HEALTH SYSTEM, INC",Hospital,Acute Care Hospital,General Acute,3.0,6,0010200003
4,"MOUNTAIN AREA PEDIATRICS ASSOCIATES, PA",Outpatient Center,Medical Group,Pediatric Medicine,NaN,10,0010200003


In [12]:
iqvia[iqvia.ME.isin(internists.IQVIA_ME)].drop_duplicates(['ME','COT_FACILITY_TYPE']).groupby(['COT_CLASSIFICATION','COT_FACILITY_TYPE']).count()[['ME']]

ME
COT_CLASSIFICATION    COT_FACILITY_TYPE                           
Academic              Medical School                          3240
                      Nursing School                             1
                      Osteopathic Medical School                27
Correctional Facility City/County/State                        408
                      Federal                                   44
                      Other                                     71
Elder Care            Adult Day Care                            70
Emergency             Ambulance                                 20
                      Rescue                                     7
Health Alliance       Organization/Service                       1
Hospital              Acute Care Hospital                   105444
                      Psychiatric Hospital                    1085
                      Rehabilitation Hospital                 1082
Insurance             Managed Care                           12310
Laboratory            Clinical Laboratory                       59
                      Other Laboratory                           2
                      Reference Laboratory                       1
Miscellaneous         Medical Bank                               1
                      Non-health Delivery                     1279
Outpatient Center     Alternative Medicine                      83
                      Clinic                                  1715
                      Health Department                        400
                      Imaging Center                           366
                      Independent Physician Practice          6210
                      Medical Group                          60243
                      Medical Spa                              145
                      Outpatient Surgi Center                  421
                      Physical Medicine and Rehabilitation     276
Pharmacy              Alternate Dispensing Site                 14
                      Closed-door                                4
                      Retail Pharmacy                           16
Research              Research Institute                       374
                      Research Laboratory                       29
Residential           Assisted Living                          284
                      Home Health                              749
                      Independent Living                        10
                      Nursing Homes                          18390
                      Residential Facilities                    57
Supply                Manufacturer                               3
Veterinary Medicine   Hospital/Clinic                           12
Work place Wellness   Employee Assistance Programs               4
                      Employer Clinic                           46
                      Executive Health Program                   3

In [16]:
iqvia[['COT_CLASSIFICATION', 'COT_FACILITY_TYPE',
       'COT_SPECIALTY']].drop_duplicates().to_csv('../../Data/IQVia/COT_Breakdown.csv', index=False)

In [61]:
family_affiliation = pd.merge(familty, iqvia, left_on='IQVIA_ME', right_on='ME')

In [62]:
dpc_family = family_affiliation[family_affiliation.TOP_CD==20]

In [81]:
dpc_family[(dpc_family.COT_CLASSIFICATION=='Emergency')|(dpc_family.COT_CLASSIFICATION=='Outpatient Center')].drop_duplicates('IQVIA_ME')

,ME_x,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,IQVIA_ME,BUSINESS_NAME,COT_CLASSIFICATION,COT_FACILITY_TYPE,COT_SPECIALTY,AFFIL_TYPE_ID,AFFIL_RANK,ME_y
0,00102000137,A,1,2.0,SHANNON C WEBB MD,WEBB,SHANNON,CLAYE,NaN,UNIT2501,...,NaN,NaN,0010200013,"NEW HORIZON FAMILY HEALTH SERVICES, INC",Outpatient Center,Medical Group,Family Practice,NaN,10,0010200013
2,00102000153,A,1,2.0,CLAUDE E SHACKELFORD MD,SHACKELFORD,CLAUDE,E,NaN,NaN,...,NaN,NaN,0010200015,VANDERBILT HEALTH AND WILLIAMSON MEDICAL CENTE...,Outpatient Center,Medical Group,Urgent Care,NaN,9,0010200015
7,00102000188,A,1,2.0,RANI M GILL MD,GILL,RANI,MANPREET,NaN,NaN,...,NaN,NaN,0010200018,HUNTSVILLE VA CLINIC,Outpatient Center,Medical Group,Multispecialty Group Practice,NaN,10,0010200018
8,00102000226,A,1,2.0,WAYNE C ANDERSON MD,ANDERSON,WAYNE,C,NaN,NaN,...,NaN,NaN,0010200022,BAYCARE MEDICAL GROUP TURLEY FAMILY HEALTH CENTER,Outpatient Center,Medical Group,Primary Care,NaN,10,0010200022
11,00102000561,A,1,1.0,CLINT A STRONG MD,STRONG,CLINT,ANDREW,NaN,JORDAN VALLEY CLINIC,...,NaN,NaN,0010200056,JORDAN VALLEY COMMUNITY HEALTH CENTER,Outpatient Center,Medical Group,Multispecialty Group Practice,NaN,10,0010200056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296451,95711110012,A,1,1.0,ALEN AJANOVIC MD,AJANOVIC,ALEN,NaN,NaN,PREMIER FAMILY PHYSICIANS,...,NaN,NaN,9571111001,PREMIER FAMILY PHYSICIANS AT SOUTHWEST MEDICAL...,Outpatient Center,Medical Group,Primary Care,NaN,10,9571111001
296453,95711870017,A,1,2.0,ERMINA MUJADZIC MD,MUJADZIC,ERMINA,NaN,NaN,NaN,...,NaN,NaN,9571187001,UNIVERSITY PROMPT CARE EVANS,Outpatient Center,Medical Group,Urgent Care,NaN,10,9571187001
296456,96501020068,A,1,1.0,MEENU PANDEY MD,PANDEY,MEENU,NaN,NaN,NaN,...,NaN,NaN,9650102006,SOUTH OKLAHOMA CITY VA CLINIC,Outpatient Center,Medical Group,Multispecialty Group Practice,NaN,6,9650102006
296457,96501040018,A,1,2.0,FARAH KHAN MD,KHAN,FARAH,NaN,NaN,NaN,...,NaN,NaN,9650104001,UNITYPOINT HEALTH MERITER DEFOREST WINDSOR CLINIC,Outpatient Center,Medical Group,Multispecialty Group Practice,NaN,10,9650104001


In [72]:
ambulatory_mes = list(family_affiliation[(family_affiliation.COT_CLASSIFICATION=='Emergency')|(family_affiliation.COT_CLASSIFICATION=='Outpatient Center')].drop_duplicates('IQVIA_ME').IQVIA_ME)

In [73]:
hospital_mes = list(family_affiliation[family_affiliation.COT_CLASSIFICATION=='Hospital'].IQVIA_ME)

In [86]:
dpc_family[(dpc_family.IQVIA_ME.isin(hospital_mes))&(dpc_family.IQVIA_ME.isin(ambulatory_mes))].drop_duplicates('IQVIA_ME')

,ME_x,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,IQVIA_ME,BUSINESS_NAME,COT_CLASSIFICATION,COT_FACILITY_TYPE,COT_SPECIALTY,AFFIL_TYPE_ID,AFFIL_RANK,ME_y
1,00102000153,A,1,2.0,CLAUDE E SHACKELFORD MD,SHACKELFORD,CLAUDE,E,NaN,NaN,...,NaN,NaN,0010200015,MAURY REGIONAL MEDICAL CENTER,Hospital,Acute Care Hospital,General Acute,3.0,6,0010200015
5,00102000188,A,1,2.0,RANI M GILL MD,GILL,RANI,MANPREET,NaN,NaN,...,NaN,NaN,0010200018,HUNTSVILLE HOSPITAL,Hospital,Acute Care Hospital,General Acute,1.0,6,0010200018
8,00102000226,A,1,2.0,WAYNE C ANDERSON MD,ANDERSON,WAYNE,C,NaN,NaN,...,NaN,NaN,0010200022,BAYCARE MEDICAL GROUP TURLEY FAMILY HEALTH CENTER,Outpatient Center,Medical Group,Primary Care,NaN,10,0010200022
11,00102000561,A,1,1.0,CLINT A STRONG MD,STRONG,CLINT,ANDREW,NaN,JORDAN VALLEY CLINIC,...,NaN,NaN,0010200056,JORDAN VALLEY COMMUNITY HEALTH CENTER,Outpatient Center,Medical Group,Multispecialty Group Practice,NaN,10,0010200056
13,00102000587,A,1,1.0,JASON D AYRES MD,AYRES,JASON,DAVID,NaN,NaN,...,NaN,NaN,0010200058,EAST GADSDEN CLINIC,Outpatient Center,Medical Group,Internal Medicine,NaN,6,0010200058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296444,95710960012,A,1,2.0,EDINA KARAHODZIC MD,KARAHODZIC,EDINA,NaN,NaN,NaN,...,NaN,NaN,9571096001,SOUTH CITY HOSPITAL,Hospital,Acute Care Hospital,General Acute,3.0,6,9571096001
296447,95710980013,A,1,2.0,AIDA JACIC MD,JACIC,AIDA,NaN,NaN,NaN,...,NaN,NaN,9571098001,FAMILY MEDICINE ASSOCIATES OF CLOVER,Outpatient Center,Medical Group,Family Practice,NaN,9,9571098001
296452,95711870017,A,1,2.0,ERMINA MUJADZIC MD,MUJADZIC,ERMINA,NaN,NaN,NaN,...,NaN,NaN,9571187001,UNIVERSITY HOSPITAL,Hospital,Acute Care Hospital,General Acute,2.0,6,9571187001
296455,96501020068,A,1,1.0,MEENU PANDEY MD,PANDEY,MEENU,NaN,NaN,NaN,...,NaN,NaN,9650102006,OKLAHOMA CITY VA MEDICAL CENTER,Hospital,Acute Care Hospital,General Acute,1.0,10,9650102006


In [42]:
internist_affiliation.drop_duplicates('IQVIA_ME')

,ME_x,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,IQVIA_ME,BUSINESS_NAME,COT_CLASSIFICATION,COT_FACILITY_TYPE,COT_SPECIALTY,AFFIL_TYPE_ID,AFFIL_RANK,ME_y
0,00102000048,A,1,2.0,NADHIWAN S MAISIAK MD,MAISIAK,NADHIWAN,SAKARIN,NaN,NaN,...,NaN,NaN,0010200004,BANNER DEL E WEBB MEDICAL CENTER,Hospital,Acute Care Hospital,General Acute,1.0,10,0010200004
1,00102000439,A,1,1.0,HEEJUNG V KANG MD,KANG,HEEJUNG,VICTORIA,NaN,NaN,...,NaN,NaN,0010200043,HUNTSVILLE HOSPITAL,Hospital,Acute Care Hospital,General Acute,3.0,6,0010200043
3,00102000447,A,1,1.0,MELINDA B HART MD,HART,MELINDA,BROWN,NaN,NaN,...,NaN,NaN,0010200044,"CULLMAN INTERNAL MEDICINE, PC",Outpatient Center,Medical Group,Internal Medicine,NaN,10,0010200044
5,00102000498,A,1,1.0,MELANIE P NICHOLLS MD,NICHOLLS,MELANIE,PIKE,NaN,NaN,...,NaN,NaN,0010200049,COVENANT CARE,Residential,Home Health,Hospice,NaN,6,0010200049
7,00102000714,A,1,2.0,RUSSELL L BEDSOLE MD,BEDSOLE,RUSSELL,LEE,NaN,NaN,...,NaN,NaN,0010200071,EXTENDICARE HEALTH AND REHABILITATION CENTER,Residential,Nursing Homes,None,6.0,8,0010200071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320847,96501910012,A,1,1.0,HANY S SALAMA MD,SALAMA,HANY,SAMIR B,NaN,NaN,...,NaN,NaN,9650191001,BARNES JEWISH WEST COUNTY HOSPITAL,Hospital,Acute Care Hospital,General Acute,3.0,6,9650191001
320858,96501930013,A,1,2.0,ANANT K SHAH MD,SHAH,ANANT,KUMAR,NaN,NaN,...,NaN,NaN,9650193001,"ACCESS HEALTH CARE PHYSICIANS, LLC",Outpatient Center,Medical Group,Internal Medicine,NaN,10,9650193001
320860,96501960036,A,1,2.0,KAMISHELE MAILA MD,MAILA,KAMISHELE,NaN,NaN,NaN,...,NaN,NaN,9650196003,ESSENTIA HEALTH FARGO,Hospital,Acute Care Hospital,General Acute,1.0,9,9650196003
320861,96501960044,A,1,2.0,MWIZA GAUSI MD,GAUSI,MWIZA,NaN,NaN,NaN,...,NaN,NaN,9650196004,VCU HEALTH COMMUNITY MEMORIAL HOSPITAL,Hospital,Acute Care Hospital,General Acute,1.0,10,9650196004


In [45]:
PPD[(PPD.PRIM_SPEC_CD=='IM')&(~PPD.IQVIA_ME.isin(list(iqvia.ME)))]

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,IQVIA_ME
80,00102000919,A,1,1.0,KATRI P PALONEN MD,PALONEN,KATRI,PAULIINA,NaN,DAVITA MEDICAL FLORIDA INC,...,33702,2211.0,C030,NaN,NaN,NaN,NaN,NaN,NaN,0010200091
105,00102001451,A,1,1.0,ROBERT R NETTLES MD,NETTLES,ROBERT,RAY,NaN,NaN,...,90404,NaN,C010,NaN,NaN,NaN,NaN,NaN,NaN,0010200145
220,00102010671,A,1,2.0,F D TRAFFANSTEDT MD,TRAFFANSTEDT,F,DARLENE H,NaN,NaN,...,35233,1502.0,C022,TRAFFANSTEDT,HARRISON,DARLENE,NaN,NaN,NaN,0010201067
406,00102020910,A,1,2.0,ALVIN V SHIH MD,SHIH,ALVIN,VICTOR,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0010202091
542,00102030702,A,1,1.0,SANDRA Y CHUNG MD,CHUNG,SANDRA,YOUNGMI,NaN,M987,...,94143,2204.0,C001,NaN,NaN,NaN,NaN,NaN,NaN,0010203070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308402,95709010019,A,1,2.0,DELVINA HASIMJA SARAQINI MD,HASIMJA SARAQINI,DELVINA,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9570901001
1308482,96501740010,A,1,2.0,ARVINDKUMAR BHAKTA MD,BHAKTA,ARVINDKUMAR,NaN,NaN,NaN,...,73134,1785.0,R037,NaN,NaN,NaN,NaN,NaN,NaN,9650174001
1308491,96501770059,A,1,2.0,ASHABEN P PATEL MD,PATEL,ASHABEN,P,NaN,NaN,...,NaN,NaN,NaN,PATEL,PARBHOOBHAI,A,NaN,NaN,NaN,9650177005
1308539,96501890046,A,1,2.0,CHRISTOPHER MWENYA MD,MWENYA,CHRISTOPHER,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9650189004


In [37]:
npi = use.get_npi_to_me()

In [127]:
len(internists)

132133

In [130]:
internists = pd.merge(internists, npi, on='ME')

In [81]:
inpatient_diagnosis_1 = pd.read_csv('../../Data/DHC/InpatientDiagnosis_1.csv')
inpatient_diagnosis_2 = pd.read_csv('../../Data/DHC/InpatientDiagnosis_2.csv')
inpatient_diagnosis = pd.concat([inpatient_diagnosis_1, inpatient_diagnosis_2])
inpatient_diagnosis.NPI = inpatient_diagnosis.NPI.astype(str)
inpatient_procedure_op_1 = pd.read_csv('../../Data/DHC/InpatientProcedureOperating_1.csv')
inpatient_procedure_op_2  = pd.read_csv('../../Data/DHC/InpatientProcedureOperating_2.csv')
inpatient_procedure_op = pd.concat([inpatient_procedure_op_1, inpatient_procedure_op_2])
inpatient_procedure_op.NPI = inpatient_procedure_op.NPI.astype(str)
inpatient_procedure_at_1 = pd.read_csv('../../Data/DHC/InpatientProcedureAttending_1.csv')
inpatient_procedure_at_2  = pd.read_csv('../../Data/DHC/InpatientProcedureAttending_2.csv')
inpatient_procedure_at = pd.concat([inpatient_procedure_at_1, inpatient_procedure_at_2])
inpatient_procedure_at.NPI = inpatient_procedure_at.NPI.astype(str)

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [101]:
outpatient_diagnosis = pd.read_csv('../../Data/DHC/OutpatientDiagnosis.csv')

In [102]:
outpatient_diagnosis.NPI = outpatient_diagnosis.NPI.astype(str)

In [123]:
inpatient_diagnosis.NPI = inpatient_diagnosis.NPI.astype(str)

In [124]:
inn_2 = pd.merge(internists, inpatient_diagnosis, on='NPI')

In [105]:
out = pd.merge(internists, outpatient_diagnosis, on='NPI')

In [112]:
internists[(internists.ME.isin(out.ME))&(~internists.ME.isin(inn.ME))]

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,IQVIA_ME,NPI
4,00102000714,A,1,2.0,RUSSELL L BEDSOLE MD,BEDSOLE,RUSSELL,LEE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0010200071,1649235995
9,00102001141,A,1,2.0,BRUCE E TAYLOR MD,TAYLOR,BRUCE,EDWARD,NaN,NaN,...,2202.0,R009,NaN,NaN,NaN,NaN,NaN,NaN,0010200114,1760410849
10,00102001214,A,1,2.0,BRIAN M SCOTT MD,SCOTT,BRIAN,MARK,NaN,NaN,...,2094.0,C007,NaN,NaN,NaN,NaN,NaN,NaN,0010200121,1851345755
13,00102001826,A,1,2.0,SHAWN J SMITH MD,SMITH,SHAWN,JASON,NaN,NaN,...,6461.0,R044,NaN,NaN,NaN,NaN,NaN,NaN,0010200182,1033271994
17,00102002334,A,1,2.0,SCOTT R FULGHAM MD,FULGHAM,SCOTT,RUSSELL,NaN,NaN,...,2959.0,C041,NaN,NaN,NaN,NaN,NaN,NaN,0010200233,1881785749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132120,96501800021,A,1,1.0,SHANTU B PATEL MD,PATEL,SHANTU,BHULABHAI,NaN,NaN,...,5102.0,C005,PATEL,B,SHANTILAL,NaN,NaN,NaN,9650180002,1851305098
132126,96501890038,A,1,NaN,BETSY E PANICKER MD,PANICKER,BETSY,ELIZABETH,NaN,NaN,...,7201.0,C027,JOSEPH,E,BETSY,JOSEPH,ELIZABETH,BETSY,9650189003,1417980699
132127,96501900017,A,1,1.0,MAMLE ANIM MD,ANIM,MAMLE,NaN,NaN,NaN,...,2643.0,C053,ONWUDIWE,ANIM,MAMLE,NaN,NaN,NaN,9650190001,1790741536
132128,96501910012,A,1,1.0,HANY S SALAMA MD,SALAMA,HANY,SAMIR B,NaN,NaN,...,5140.0,C069,NaN,NaN,NaN,NaN,NaN,NaN,9650191001,1619064656


In [125]:
internists[(internists.ME.isin(inn_2.ME))]

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,IQVIA_ME,NPI
0,00102000048,A,1,2.0,NADHIWAN S MAISIAK MD,MAISIAK,NADHIWAN,SAKARIN,NaN,NaN,...,5282.0,C045,NaN,NaN,NaN,NaN,NaN,NaN,0010200004,1528054913
2,00102000447,A,1,1.0,MELINDA B HART MD,HART,MELINDA,BROWN,NaN,NaN,...,689.0,R024,NaN,NaN,NaN,NaN,NaN,NaN,0010200044,1174528020
6,00102000803,A,1,2.0,EUGENE L LIU MD,LIU,EUGENE,LIN,NaN,NaN,...,1051.0,R010,NaN,NaN,NaN,NaN,NaN,NaN,0010200080,1770584930
8,00102001095,A,1,2.0,BROOK WATTS MD,WATTS,BROOK,NaN,NaN,NaN,...,1900.0,C005,NaN,NaN,NaN,NaN,NaN,NaN,0010200109,1497775563
14,00102001885,A,1,2.0,JANINE R ELLIS MD,ELLIS,JANINE,ROBINSON,NaN,NaN,...,1701.0,C006,ELLIS,LETRICIA,JANINE,ROBINSON,LETRICIA,JANINE,0010200188,1093896516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132123,96501850036,A,1,1.0,RASHID A DALAL MD,DALAL,RASHID,AHMED M,NaN,NaN,...,5369.0,C026,NaN,NaN,NaN,NaN,NaN,NaN,9650185003,1265434815
132128,96501910012,A,1,1.0,HANY S SALAMA MD,SALAMA,HANY,SAMIR B,NaN,NaN,...,5140.0,C069,NaN,NaN,NaN,NaN,NaN,NaN,9650191001,1619064656
132130,96501960036,A,1,2.0,KAMISHELE MAILA MD,MAILA,KAMISHELE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9650196003,1720144272
132131,96501960044,A,1,2.0,MWIZA GAUSI MD,GAUSI,MWIZA,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9650196004,1750688651


In [158]:
all_in = internists[(internists.NPI.isin(inpatient_procedure_at.NPI))|(internists.NPI.isin(inpatient_procedure_op.NPI))|(internists.NPI.isin(inpatient_diagnosis.NPI))]

In [126]:
len(inpatient_diagnosis)

195943

In [133]:
len(internists)

132133

In [137]:
sample = pd.read_csv('../../Data/DHC/OutpatientProcedureAttending_sample.csv')
sample.NPI = sample.NPI.astype(str)
internal = pd.read_csv('../../Data/DHC/OutpatientProcedureAttending_internal.csv')
internal.NPI = internal.NPI.astype(str)

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [149]:
sample[sample.NPI.isin(internists.NPI)].groupby('Primary Specialty').count().sort_values('NPI', ascending=False)

,NPI,Primary Hospital Affiliation,State,Total Pmts,Avg Pmt/Procedure,Total Charges,Avg Charge/Procedure,Total # of Procedures
Primary Specialty,,,,,,,,
Internal Medicine,13454,13293,13454,13454,13454,13454,13454,13454
Hospitalist,3686,3667,3686,3686,3686,3686,3686,3686
Emergency Medicine,586,584,586,586,586,586,586,586
Nephrology,484,469,484,484,484,484,484,484
Gastroenterology,272,271,272,272,272,272,272,272
Cardiology - Cardiologist,259,258,259,259,259,259,259,259
Oncology - Hematology/Oncology,180,180,180,180,180,180,180,180
Oncology - Medical Oncology,121,121,121,121,121,121,121,121
Pulmonary Disease,110,110,110,110,110,110,110,110


In [142]:
internal[internal.NPI.isin(internists.NPI)]

,NPI,Last Name,State,Primary Hospital Affiliation,Total Pmts,Avg Pmt/Procedure,Total Charges,Avg Charge/Procedure,Total # of Procedures
13,1861582942,Jones,OH,"Akron General (Akron, OH)","$2,186,594",$25,"$4,699,067",$54,"86,354"
14,1700840980,White,AR,"Baxter Regional Medical Center (Mountain Home,...","$2,184,059",$395,"$6,347,282","$1,148","5,531"
25,1487748356,Nassif,FL,"Cape Coral Hospital (Cape Coral, FL)","$1,750,137",$230,"$14,292,630","$1,876","7,617"
37,1952337859,Jarrett,NY,Holy Name Medical Center (AKA Holy Name Health...,"$1,405,788",$29,"$3,905,312",$80,"48,968"
39,1902284680,Patel,NJ,"Community Medical Center (Toms River, NJ)","$1,395,478",$109,"$10,564,408",$827,"12,776"
...,...,...,...,...,...,...,...,...,...
89002,1477533602,Badve,MS,Pascagoula Hospital (FKA Singing River Hospita...,$0,NaN,$780,NaN,NaN
89004,1932360492,Dahya,CA,Kaiser Permanente - South San Francisco Medica...,$0,NaN,"$1,433",NaN,NaN
89007,1972675825,Byrne,FL,AdventHealth New Smyrna Beach (FKA Florida Hos...,$0,NaN,"$1,068",NaN,NaN
89008,1184641268,Osburn,SC,NaN,$0,NaN,$464,NaN,NaN


In [150]:
len(internal)

89011

In [151]:
outpatient_att = pd.read_csv('../../Data/DHC/OutpatientProcedureAttending.csv')
outpatient_att.NPI = outpatient_att.NPI.astype(str)
outpatient_op = pd.read_csv('../../Data/DHC/OutpatientProcedureOperating.csv')
outpatient_op.NPI = outpatient_op.NPI.astype(str)

In [155]:
outpatient_op[outpatient_op.NPI.isin(internists.NPI)]

,NPI,First Name,Primary Specialty,Primary Hospital Affiliation,Total Pmts,Avg Pmt/Procedure,Total Charges,Avg Charge/Procedure,Total # of Procedures
16,1902031735,Derrick,Internal Medicine,ShorePoint Health Port Charlotte (FKA Bayfront...,"$926,247",$359,"$8,892,128","$3,448","2,579"
17,1164628079,Ania,Internal Medicine,Mount Sinai Medical Center (AKA the Mount Sina...,"$913,249",$42,"$4,547,720",$207,"21,923"
26,1982752465,Mohammed,Internal Medicine,Northridge Hospital Medical Center (Northridge...,"$735,440",$175,"$5,509,329","$1,314","4,194"
33,1811905516,Francis,Internal Medicine,UMass Memorial Health - Harrington (FKA Harrin...,"$684,985",$35,"$1,485,208",$76,"19,476"
35,1356443675,Patrick,Internal Medicine,The University of Kansas Hospital (Kansas City...,"$670,382","$1,106","$1,843,456","$3,042",606
...,...,...,...,...,...,...,...,...,...
64471,1780668350,Jeffrey,Internal Medicine,"Speare Memorial Hospital (Plymouth, NH)",$0,NaN,"$3,665",NaN,NaN
64472,1841678091,Suzin,Hospitalist,Prisma Health Greenville Memorial Hospital (Gr...,$0,NaN,$42,NaN,NaN
64473,1194779454,Maria,Internal Medicine,The Valley Hospital (AKA Valley Health System)...,$0,NaN,"$1,512",NaN,NaN
64474,1467411769,Susan,Internal Medicine,The University of Vermont Medical Center (FKA ...,$0,$0,"$10,522",$263,40


In [154]:
outpatient_att[outpatient_att.NPI.isin(internists.NPI)]

,NPI,Primary Specialty,Primary Hospital Affiliation,Total Pmts,Avg Pmt/Procedure,Total Charges,Avg Charge/Procedure,Total # of Procedures
13,1861582942,Internal Medicine,"Akron General (Akron, OH)","$2,186,594",$25,"$4,699,067",$54,"86,354"
14,1700840980,Internal Medicine,"Baxter Regional Medical Center (Mountain Home,...","$2,184,059",$395,"$6,347,282","$1,148","5,531"
18,1255598397,Hospitalist,"Yale New Haven Hospital (New Haven, CT)","$2,046,323",$32,"$9,297,231",$145,"63,982"
26,1487748356,Internal Medicine,"Cape Coral Hospital (Cape Coral, FL)","$1,750,137",$230,"$14,292,630","$1,876","7,617"
29,1215227392,Hospitalist,Hill Country Memorial Hospital (Fredericksburg...,"$1,616,261",$401,"$5,684,853","$1,410","4,031"
...,...,...,...,...,...,...,...,...
112320,1740346865,Internal Medicine,NaN,$0,NaN,$93,NaN,NaN
112321,1922193572,Internal Medicine,Virginia Hospital Center - Arlington (Arlingto...,$0,NaN,$76,NaN,NaN
112322,1275657215,Internal Medicine,NaN,$0,NaN,$510,NaN,NaN
112323,1386638732,Internal Medicine,"StoneSprings Hospital Center (Dulles, VA)",$0,NaN,"$1,551",NaN,NaN


In [157]:
all_out = internists[(internists.NPI.isin(outpatient_att.NPI))|(internists.NPI.isin(outpatient_op.NPI))|(internists.NPI.isin(outpatient_diagnosis.NPI))]

In [164]:
internists[(internists.NPI.isin(all_out.NPI))|(internists.NPI.isin(all_in.NPI))]

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,IQVIA_ME,NPI
0,00102000048,A,1,2.0,NADHIWAN S MAISIAK MD,MAISIAK,NADHIWAN,SAKARIN,NaN,NaN,...,5282.0,C045,NaN,NaN,NaN,NaN,NaN,NaN,0010200004,1528054913
2,00102000447,A,1,1.0,MELINDA B HART MD,HART,MELINDA,BROWN,NaN,NaN,...,689.0,R024,NaN,NaN,NaN,NaN,NaN,NaN,0010200044,1174528020
4,00102000714,A,1,2.0,RUSSELL L BEDSOLE MD,BEDSOLE,RUSSELL,LEE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0010200071,1649235995
6,00102000803,A,1,2.0,EUGENE L LIU MD,LIU,EUGENE,LIN,NaN,NaN,...,1051.0,R010,NaN,NaN,NaN,NaN,NaN,NaN,0010200080,1770584930
8,00102001095,A,1,2.0,BROOK WATTS MD,WATTS,BROOK,NaN,NaN,NaN,...,1900.0,C005,NaN,NaN,NaN,NaN,NaN,NaN,0010200109,1497775563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132128,96501910012,A,1,1.0,HANY S SALAMA MD,SALAMA,HANY,SAMIR B,NaN,NaN,...,5140.0,C069,NaN,NaN,NaN,NaN,NaN,NaN,9650191001,1619064656
132129,96501930013,A,1,2.0,ANANT K SHAH MD,SHAH,ANANT,KUMAR,NaN,NaN,...,3501.0,C069,SHAH,M,ANANTKUMAR,NaN,NaN,NaN,9650193001,1356378905
132130,96501960036,A,1,2.0,KAMISHELE MAILA MD,MAILA,KAMISHELE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9650196003,1720144272
132131,96501960044,A,1,2.0,MWIZA GAUSI MD,GAUSI,MWIZA,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9650196004,1750688651


In [63]:
outpatient_1 = pd.read_csv('../../Data/DHC/OutpatientDiagnosisWest.csv')
outpatient_1.NPI = outpatient_1.NPI.astype(str)
outpatient_2 = pd.read_csv('../../Data/DHC/OutpatientDiagnosisSouthwest.csv')
outpatient_2.NPI = outpatient_2.NPI.astype(str)
outpatient_3 = pd.read_csv('../../Data/DHC/OutpatientDiagnosisNortheast.csv')
outpatient_3.NPI = outpatient_3.NPI.astype(str)
outpatient_4 = pd.read_csv('../../Data/DHC/OutpatientDiagnosisTerritoriesUnknown.csv')
outpatient_4.NPI = outpatient_4.NPI.astype(str)
outpatient_5 = pd.read_csv('../../Data/DHC/OutpatientDiagnosisMidwest.csv')
outpatient_5.NPI = outpatient_5.NPI.astype(str)
outpatient_6 = pd.read_csv('../../Data/DHC/OutpatientDiagnosisSoutheast.csv')
outpatient_6.NPI = outpatient_6.NPI.astype(str)
outpatient_d = pd.read_csv('../../Data/DHC/OutpatientDiagnosisInternalSpec.csv')
outpatient_d.NPI = outpatient_d.NPI.astype(str)
outpatient_f = pd.read_csv('../../Data/DHC/OutpatientDiagnosisFamily.csv')
outpatient_f.NPI = outpatient_f.NPI.astype(str)
outpatient_diagnosis = pd.concat([outpatient_1,outpatient_2,outpatient_3,outpatient_4,outpatient_5,outpatient_6,outpatient_d,outpatient_f]).drop_duplicates('NPI')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [64]:
outpatient_1 = pd.read_csv('../../Data/DHC/OutpatientProcedureAttWest.csv')
outpatient_1.NPI = outpatient_1.NPI.astype(str)
outpatient_2 = pd.read_csv('../../Data/DHC/OutpatientProcedureAttSouthwest.csv')
outpatient_2.NPI = outpatient_2.NPI.astype(str)
outpatient_3 = pd.read_csv('../../Data/DHC/OutpatientProcedureAttNortheast.csv')
outpatient_3.NPI = outpatient_3.NPI.astype(str)
outpatient_4 = pd.read_csv('../../Data/DHC/OutpatientProcedureAttTerritoriesUnknown.csv')
outpatient_4.NPI = outpatient_4.NPI.astype(str)
outpatient_5 = pd.read_csv('../../Data/DHC/OutpatientProcedureAttMidwest.csv')
outpatient_5.NPI = outpatient_5.NPI.astype(str)
outpatient_6 = pd.read_csv('../../Data/DHC/OutpatientProcedureAttSoutheast.csv')
outpatient_6.NPI = outpatient_6.NPI.astype(str)
outpatient_d = pd.read_csv('../../Data/DHC/OutpatientProcedureAttInternalSpec.csv')
outpatient_d.NPI = outpatient_d.NPI.astype(str)
outpatient_f = pd.read_csv('../../Data/DHC/OutpatientProcedureAttFamily.csv')
outpatient_f.NPI = outpatient_f.NPI.astype(str)
outpatient_procedure_at = pd.concat([outpatient_1,outpatient_2,outpatient_3,outpatient_4,outpatient_5,outpatient_6,outpatient_d,outpatient_f]).drop_duplicates('NPI')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [65]:
outpatient_1 = pd.read_csv('../../Data/DHC/OutpatientProcedureOpWest.csv')
outpatient_1.NPI = outpatient_1.NPI.astype(str)
outpatient_2 = pd.read_csv('../../Data/DHC/OutpatientProcedureOpSouthwest.csv')
outpatient_2.NPI = outpatient_2.NPI.astype(str)
outpatient_3 = pd.read_csv('../../Data/DHC/OutpatientProcedureOpNortheast.csv')
outpatient_3.NPI = outpatient_3.NPI.astype(str)
outpatient_4 = pd.read_csv('../../Data/DHC/OutpatientProcedureOpTerritoriesUnknown.csv')
outpatient_4.NPI = outpatient_4.NPI.astype(str)
outpatient_5 = pd.read_csv('../../Data/DHC/OutpatientProcedureOpMidwest.csv')
outpatient_5.NPI = outpatient_5.NPI.astype(str)
outpatient_6 = pd.read_csv('../../Data/DHC/OutpatientProcedureOpSoutheast.csv')
outpatient_6.NPI = outpatient_6.NPI.astype(str)
outpatient_procedure_op = pd.concat([outpatient_1,outpatient_2,outpatient_3,outpatient_4,outpatient_5,outpatient_6]).drop_duplicates('NPI')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


In [61]:
internists[(internists.NPI.isin(outpatient_diagnosis.NPI))|(internists.NPI.isin(outpatient_d.NPI))]

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,IQVIA_ME,NPI
3,00102000048,A,1,2.0,NADHIWAN S MAISIAK MD,MAISIAK,NADHIWAN,SAKARIN,NaN,NaN,...,5282.0,C045,NaN,NaN,NaN,NaN,NaN,NaN,0010200004,1528054913
40,00102000447,A,1,1.0,MELINDA B HART MD,HART,MELINDA,BROWN,NaN,NaN,...,689.0,R024,NaN,NaN,NaN,NaN,NaN,NaN,0010200044,1174528020
66,00102000714,A,1,2.0,RUSSELL L BEDSOLE MD,BEDSOLE,RUSSELL,LEE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0010200071,1649235995
73,00102000803,A,1,2.0,EUGENE L LIU MD,LIU,EUGENE,LIN,NaN,NaN,...,1051.0,R010,NaN,NaN,NaN,NaN,NaN,NaN,0010200080,1770584930
84,00102001095,A,1,2.0,BROOK WATTS MD,WATTS,BROOK,NaN,NaN,NaN,...,1900.0,C005,NaN,NaN,NaN,NaN,NaN,NaN,0010200109,1497775563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081078,96501910012,A,1,1.0,HANY S SALAMA MD,SALAMA,HANY,SAMIR B,NaN,NaN,...,5140.0,C069,NaN,NaN,NaN,NaN,NaN,NaN,9650191001,1619064656
1081081,96501930013,A,1,2.0,ANANT K SHAH MD,SHAH,ANANT,KUMAR,NaN,NaN,...,3501.0,C069,SHAH,M,ANANTKUMAR,NaN,NaN,NaN,9650193001,1356378905
1081086,96501960036,A,1,2.0,KAMISHELE MAILA MD,MAILA,KAMISHELE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9650196003,1720144272
1081087,96501960044,A,1,2.0,MWIZA GAUSI MD,GAUSI,MWIZA,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9650196004,1750688651


In [52]:
outpatient_diagnosis[outpatient_diagnosis.NPI.isin(familty.NPI)].groupby('Primary Specialty').count().sort_values('NPI', ascending=False)

,Avg Charge/Claim,Avg Payment/Claim,NPI,Primary Hospital Affiliation,State,Total # of Claims,Total Charges,Total Pmts
Primary Specialty,,,,,,,,
Family Practice,22739,22739,36602,35237,28104,22739,36602,36602
Emergency Medicine,983,983,1227,1186,775,983,1227,1227
Hospitalist,657,657,1071,1064,808,657,1071,1071
Internal Medicine,341,341,613,591,528,341,613,613
General Practice,257,257,512,471,398,257,512,512
...,...,...,...,...,...,...,...,...
Psychology - Psychologist,1,1,1,1,1,1,1,1
Cardiology - Peripheral Vascular Disease,1,1,1,1,0,1,1,1
Cardiology - Cardiac Electrophysiology,1,1,1,1,1,1,1,1


In [103]:
internists[internists.NPI.isin(inpatient_procedure_at.NPI)]

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,IQVIA_ME,NPI
3,00102000048,A,1,2.0,NADHIWAN S MAISIAK MD,MAISIAK,NADHIWAN,SAKARIN,NaN,NaN,...,5282.0,C045,NaN,NaN,NaN,NaN,NaN,NaN,0010200004,1528054913
40,00102000447,A,1,1.0,MELINDA B HART MD,HART,MELINDA,BROWN,NaN,NaN,...,689.0,R024,NaN,NaN,NaN,NaN,NaN,NaN,0010200044,1174528020
73,00102000803,A,1,2.0,EUGENE L LIU MD,LIU,EUGENE,LIN,NaN,NaN,...,1051.0,R010,NaN,NaN,NaN,NaN,NaN,NaN,0010200080,1770584930
84,00102001095,A,1,2.0,BROOK WATTS MD,WATTS,BROOK,NaN,NaN,NaN,...,1900.0,C005,NaN,NaN,NaN,NaN,NaN,NaN,0010200109,1497775563
122,00102001885,A,1,2.0,JANINE R ELLIS MD,ELLIS,JANINE,ROBINSON,NaN,NaN,...,1701.0,C006,ELLIS,LETRICIA,JANINE,ROBINSON,LETRICIA,JANINE,0010200188,1093896516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081060,96501850036,A,1,1.0,RASHID A DALAL MD,DALAL,RASHID,AHMED M,NaN,NaN,...,5369.0,C026,NaN,NaN,NaN,NaN,NaN,NaN,9650185003,1265434815
1081078,96501910012,A,1,1.0,HANY S SALAMA MD,SALAMA,HANY,SAMIR B,NaN,NaN,...,5140.0,C069,NaN,NaN,NaN,NaN,NaN,NaN,9650191001,1619064656
1081086,96501960036,A,1,2.0,KAMISHELE MAILA MD,MAILA,KAMISHELE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9650196003,1720144272
1081087,96501960044,A,1,2.0,MWIZA GAUSI MD,GAUSI,MWIZA,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9650196004,1750688651


In [106]:
inpatient_in = internists[(internists.NPI.isin(inpatient_procedure_at.NPI))|(internists.NPI.isin(inpatient_procedure_op.NPI))|(internists.NPI.isin(inpatient_diagnosis.NPI))]

In [107]:
outpatient_in = internists[(internists.NPI.isin(outpatient_procedure_at.NPI))|(internists.NPI.isin(outpatient_procedure_op.NPI))|(internists.NPI.isin(outpatient_diagnosis.NPI))]

In [114]:
XX = internists[(internists.NPI.isin(inpatient_in.NPI))|(internists.NPI.isin(outpatient_in.NPI))]

In [115]:
IN = PPD[PPD.PRIM_SPEC_CD=='IM']

In [116]:
IN[~(IN.ME.isin(XX.ME))]

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_STATE,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
39,00102000439,A,1,1.0,HEEJUNG V KANG MD,KANG,HEEJUNG,VICTORIA,NaN,NaN,...,AL,35801,3762.0,C018,NaN,NaN,NaN,NaN,NaN,NaN
45,00102000498,A,1,1.0,MELANIE P NICHOLLS MD,NICHOLLS,MELANIE,PIKE,NaN,NaN,...,AL,36301,1314.0,C024,PIKE,FRASER,MELANIE,NaN,NaN,NaN
72,00102000773,A,1,1.0,ANITA R SMITH MD,SMITH,ANITA,ROCHELLE,NaN,NaN,...,GA,30907,410.0,C048,NaN,NaN,NaN,NaN,NaN,NaN
80,00102000919,A,1,1.0,KATRI P PALONEN MD,PALONEN,KATRI,PAULIINA,NaN,DAVITA MEDICAL FLORIDA INC,...,FL,33702,2211.0,C030,NaN,NaN,NaN,NaN,NaN,NaN
105,00102001451,A,1,1.0,ROBERT R NETTLES MD,NETTLES,ROBERT,RAY,NaN,NaN,...,CA,90404,NaN,C010,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308610,96501770059,A,1,2.0,ASHABEN P PATEL MD,PATEL,ASHABEN,P,NaN,NaN,...,NaN,NaN,NaN,NaN,PATEL,PARBHOOBHAI,A,NaN,NaN,NaN
1308637,96501830051,A,1,2.0,CHRISTOPHER T DA COSTA MD,DA COSTA,CHRISTOPHER,T K,NaN,NaN,...,NaN,NaN,NaN,NaN,ABOAGYE-DA COSTA,NaN,CHRIS,DA COSTA,K,CHRISTOPHER
1308643,96501850052,A,1,2.0,DARSHAN S SOHAL MD,SOHAL,DARSHAN,S,NaN,NaN,...,VA,20190,5286.0,C017,SOHAL,SINGH,DARSHAN,NaN,NaN,NaN
1308658,96501890046,A,1,2.0,CHRISTOPHER MWENYA MD,MWENYA,CHRISTOPHER,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
